<font style="font-size: 3rem; color: darkviolet"> Character level language model - Dinosaurus Island </font> 

AA - DEL - 2023/24 - TP4 - 3h

Author: Francesca Galassi

*This assignment is inspired by the Deep Learning course on Coursera by Andrew Ng, Stanford University, for which we are thankful.*

**Submit this notebook with your solutions, answers and observations.**

Welcome to Dinosaurus Island! Dinosaurs existed 65 million years ago, and now they're back.

Researchers are creating new breeds of dinosaurs and bringing them to life on Earth. Your task is to give names to these creatures. But be careful - if a dinosaur doesn't like its name, it might get angry!

<table>
<td>
<img src="images/dino.jpg" style="width:150;height:200px;">

</td>

</table>

Luckily, you're equipped with some deep learning. Your assistant has collected a list of all the dinosaur names they could find and compiled them into this [dataset](dinos.txt). To create new dinosaur names, you will build a character-level language model to generate new names. Your algorithm will learn different name patterns and randomly generate new names. Hopefully, this algorithm will keep you and your team safe from the dinosaurs' anger!

By completing this assignment, you'll be able to:

* Prepare text data for processing with an RNN 
* Sample novel sequences with an RNN
* Understand and fix issues with gradients in RNNs

## Table of Contents

- [1 - Problem Statement](#1)
    - [1.1 - Dataset and Preprocessing](#1-1)
    - [1.2 - Overview of the Model](#1-2)
- [2 - Building Blocks of the Model](#2)
    - [2.1 - Clipping the Gradients in the Optimization Loop](#2-1)
        - [Exercise 1](#ex-1)
    - [2.2 - Sampling](#2-2)
        - [Exercise 2](#ex-2)
- [3 - The Language Model](#3)
    - [3.1 - Gradient Descent](#3-1)
        - [Exercise 3](#ex-3)
    - [3.2 - Training the Model](#3-2)
        - [Exercise 4](#ex-4)

In [1]:
import numpy as np
from utils import *
import random
import pprint
import copy

<a name='1'></a>
## <font color='darkviolet'> 1 - Problem Statement

<a name='1-1'></a>
### <font color='darkviolet'> 1.1 - Dataset and Preprocessing

Run the following code to read the dataset of dinosaur names, convert it to lowercase, and create a list of unique characters (such as a-z). Then, we'll compute the dataset size and the size of our vocabulary.

In [2]:
data = open('dinos.txt', 'r').read()
data= data.lower() # Convert to lowercase
chars = list(set(data)) # Create a list of unique characters
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.'  % (data_size, vocab_size)) # Compute dataset and vocabulary size

There are 19909 total characters and 27 unique characters in your data.


The characters include a-z (26 characters) plus the "\n" (newline character). The newline character "\n" indicates the end of a dinosaur name, similar to the <EOS> token discussed in the lecture.
    
We'll create two dictionaries for the RNN to handle characters: 
* `char_to_ix`: this dictionary, i.e., a hash table, maps each character to an index from 0-26.
* `ix_to_char`: this dictionary maps each index back to the corresponding character. 
    -  This will help you figure out which index corresponds to which character in the probability distribution output of the softmax layer. 

In [3]:
chars = sorted(chars) # Sort the characters
print(chars)

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char_to_ix = { ch:i for i,ch in enumerate(chars) } # Map characters to indices
ix_to_char = { i:ch for i,ch in enumerate(chars) } # Map indices to characters
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(char_to_ix)
#pp.pprint(ix_to_char)

<a name='1-2'></a>
### <font color='darkviolet'> 1.2 - Overview of the Model<a name='1-2'></a>

Your model will have the following structure: 

- Initialize parameters: set up the initial weights and biases.
- Run the optimization loop:
    - Forward propagation to compute the loss function.
    - Backward propagation to compute the gradients with respect to the loss function.
    - Clip the gradients to prevent exploding gradients.
    - Update parameters using the gradient descent update rule.
- Return the learned parameters. 
    

During training, at each time-step, the Recurrent Neural Network (RNN) tries to predict the next character based on the previous characters it has seen in the sequence.
    
<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center><font color='purple'><b>Figure 1</b>: Recurrent Neural Network  </center></caption>
    
If we represent the input sequence as $\mathbf{X} = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$, where each $x^{\langle t \rangle}$ is a character from the training set, the output sequence $\mathbf{Y} = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ is the same list of characters but shifted one character forward.
At every time-step $t$, $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$, meaning that the prediction at time $t$ is the same as the input at time $t + 1$.

<a name='2'></a>
## <font color='darkviolet'> 2 - Building Blocks of the Model

<a name='2-1'></a>
### <font color='darkviolet'> 2.1 - Clipping the Gradients in the Optimization Loop
    
In this section, you will implement the `clip` function to prevent exploding gradients during training.

#### Exploding gradients
* When gradients become large, they're called "exploding gradients."  
* Exploding gradients make the training process more difficult, because the updates may be so large that they "overshoot" the optimal values during back propagation.

Recall that the overall training loop structure consists of:
* forward pass, 
* cost computation, 
* backward pass, 
* parameter update. 

Before updating the parameters, you'll perform gradient clipping to ensure that your gradients don't "explode".

<a name='ex-1'></a>
### <font color='blue'> Exercise 1
    
In this exercise, you will implement a function `clip` that takes a dictionary of gradients and returns a clipped version if needed.

* There are different ways to clip gradients.You will use a simple element-wise clipping procedure, where each element of the gradient vector is clipped to a range of [-N, N]. 
* For example, if N=10:
    - The range is [-10, 10].
    - If any gradient component is greater than 10, it is set to 10.
    - If any gradient component is less than -10, it is set to -10. 
    - Components between -10 and 10 keep their original values.

Visualization of gradient descent with and without gradient clipping, in a case where the network is running into "exploding gradient" problems:
    
<img src="images/clip.png" style="width:350;height:125px;">
<caption><center><font color='purple'><b>Figure 2</b>: The "exploding gradient" problem. </center></caption>
   
Complete the implementation of the function `clip`. The function `clip` takes in a maximum threshold and returns the clipped versions of the gradients. 
You will use [numpy.clip](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html) function for this purpose.
    
- Note: It's important to use the `out` parameter to update the gradients in-place. If you don't use `out`, the clipped variable is stored in a separate variable, not updating the gradient variables `dWax`, `dWaa`, `dWya`, `db`, `dby`.

In [5]:
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- the threshold value; gradients above this will be set to this value, and gradients below -maxValue will be set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    gradients = copy.deepcopy(gradients)
    
    # Extract individual gradients
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # Clip gradients to mitigate exploding gradients
    for gradient in None:
        np.clip(None, None, None, out=None)
    
    # Update the gradients dictionary
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [6]:
# Test
def clip_test(target, mValue):
    print(f"\nGradients for mValue={mValue}")
    np.random.seed(3)
    dWax = np.random.randn(5, 3) * 10
    dWaa = np.random.randn(5, 5) * 10
    dWya = np.random.randn(2, 5) * 10
    db = np.random.randn(5, 1) * 10
    dby = np.random.randn(2, 1) * 10
    gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}

    gradients2 = target(gradients, mValue)
    print("gradients[\"dWaa\"][1][2] =", gradients2["dWaa"][1][2])
    print("gradients[\"dWax\"][3][1] =", gradients2["dWax"][3][1])
    print("gradients[\"dWya\"][1][2] =", gradients2["dWya"][1][2])
    print("gradients[\"db\"][4] =", gradients2["db"][4])
    print("gradients[\"dby\"][1] =", gradients2["dby"][1])
    
    for grad in gradients2.keys():
        valuei = gradients[grad]
        valuef = gradients2[grad]
        mink = np.min(valuef)
        maxk = np.max(valuef)
        assert mink >= -abs(mValue), f"Problem with {grad}. Set a_min to -mValue in the np.clip call"
        assert maxk <= abs(mValue), f"Problem with {grad}.Set a_max to mValue in the np.clip call"
        index_not_clipped = np.logical_and(valuei <= mValue, valuei >= -mValue)
        assert np.all(valuei[index_not_clipped] == valuef[index_not_clipped]), f" Problem with {grad}. Some values that should not have changed, changed during the clipping process."
    
    print("\033[92mAll tests passed!\x1b[0m")
    
clip_test(clip, 10)
clip_test(clip, 5)


Gradients for mValue=10


TypeError: 'NoneType' object is not iterable

**Expected values**
```
Gradients for mValue=10
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]

Gradients for mValue=5
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [5.]
gradients["dby"][1] = [5.]
```

<a name='2-2'></a>
### <font color='darkviolet'> 2.2 - Sampling

Assume the model has already been trained on a dataset of dinosaur names, and now you would like to generate new text, character by character. Here's how the generation process works:
    
<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center><font color='purple'><b>Figure 3</b>: You feed the initial input $x^{\langle 1\rangle} = \vec{0}$ into the model at the first time-step, and the network samples one character at a time. </center></caption>

<a name='ex-2'></a>
### <font color='blue'> Exercise 2 

Implement the `sample` function to sample characters. Follow these 4 steps:

- **Step 1**: Initialize with a "dummy" vector of zeros $x^{\langle 1 \rangle} = \vec{0}$. 
    - This is the default input before any characters are generated.
    - Set $a^{\langle 0 \rangle} = \vec{0}$ as well.

- **Step 2**: Perform one step of forward propagation to compute $a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$. Here are the equations:


*Hidden state* : 
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

*Prediction:*
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle }) = softmax(W_{ya}  a^{\langle t + 1 \rangle } + b_y) \tag{2}$$

Details about $\hat{y}^{\langle t+1 \rangle }$:
   - $\hat{y}^{\langle t+1 \rangle }$ is a softmax probability vector, meaning its entries are between 0 and 1, and they sum to 1. 
   - $\hat{y}^{\langle t+1 \rangle}_i$ represents the probability that the character indexed by "i" is the next character.  
   - A `softmax()` function is provided for you to use.

*Additional Hints*

- $x^{\langle 1 \rangle}$ is `x` in the code. When creating the one-hot vector, make a numpy array of zeros, with the number of rows equal to the number of unique characters, and the number of columns equal to one. It's a 2D array and not a 1D array.
- $a^{\langle 0 \rangle}$ is `a_prev` in the code.  It is a numpy array of zeros, where the number of rows is $n_{a}$, and number of columns is 1. $n_{a}$ is retrieved by getting the number of columns in $W_{aa}$ (the numbers need to match in order for the matrix multiplication $W_{aa}a^{\langle t \rangle}$ to work.
- Official documentation for [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) and [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)

- **Step 3**: Sampling. 
    - Now that you have $y^{\langle t+1 \rangle}$, you want to select the next letter in the dinosaur name. If you select the most probable, the model will always generate the same result given a starting letter. Use [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html) to select a next letter that is *likely*, but not always the same.
    - Pick the next character's **index** according to the probability distribution specified by $\hat{y}^{\langle t+1 \rangle }$. 
    - This means that if $\hat{y}^{\langle t+1 \rangle }_i = 0.16$, you will pick the index "i" with 16% probability. 

- **Step 4**: Updating $x^{\langle t \rangle }$ .
    - The last step in `sample()` is to update the variable `x`, which currently stores $x^{\langle t \rangle }$, with the value of $x^{\langle t + 1 \rangle }$. 
    - To represent $x^{\langle t + 1 \rangle }$, create a one-hot vector corresponding to the character chosen as the prediction. 
    - Forward propagate $x^{\langle t + 1 \rangle }$ in Step 1 and repeate the process until a `"\n"` character is encountered, indicating the end of the dinosaur name. 

In [ ]:
def sample(parameters, char_to_ix):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- Python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- Python dictionary mapping each character to an index.

    Returns:
    indices -- A list of length n containing the indices of the sampled characters.
    """
   
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Step 1: Initialize input vector and hidden state
    x = None # One-hot vector for the first character
    a_prev = None # Initial hidden state
    
    # Initialize list to store sampled indices
    indices = []
    
    # Initialize index
    idx = -1 
    
    # Define the newline character index
    newline_character = char_to_ix['\n']
    counter = 0
    
    # Sampling loop
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagation to compute probabilities
        a = None
        z = None
        y = None # 2D array containing the probability distribution for each character 
        
        # Step 3: Sample a character index from the probability distribution
        idx = None

        # Append the index to the list of sampled indices
        indices.append(idx)
        
        # Step 4: Update the input vector for the next iteration
        x = None
        x[idx] = None
        
        # Update the previous hidden state
        a_prev = a
        
        # Update counter
        counter += 1

    # Add newline character index if the loop was stopped
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [ ]:
def sample_test(target):
    np.random.seed(24)
    _, n_a = 20, 100
    Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
    b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

    indices = target(parameters, char_to_ix)
    print("Sampling:")
    print("list of sampled indices:\n", indices)
    print("list of sampled characters:\n", [ix_to_char[i] for i in indices])
    
    assert len(indices) < 52, "Indices length must be smaller than 52"
    assert indices[-1] == char_to_ix['\n'], "All samples must end with \\n"
    assert min(indices) >= 0 and max(indices) < len(char_to_ix), f"Sampled indexes must be between 0 and len(char_to_ix)={len(char_to_ix)}"
    assert np.allclose(indices[0:6], [23, 16, 26, 26, 24, 3]), "Wrong values"
    
    print("\033[92mAll tests passed!")

sample_test(sample)

**Expected output**
```
Sampling:
list of sampled indices:
 [23, 16, 26, 26, 24, 3, 21, 1, 7, 24, 15, 3, 25, 20, 6, 13, 10, 8, 20, 12, 2, 0]
list of sampled characters:
 ['w', 'p', 'z', 'z', 'x', 'c', 'u', 'a', 'g', 'x', 'o', 'c', 'y', 't', 'f', 'm', 'j', 'h', 't', 'l', 'b', '\n']
```

<a name='3'></a>
## <font color='darkviolet'> 3 - The Language Model 

It's time to build the character-level language model for text generation! 


<a name='3-1'></a>
### <font color='darkviolet'> 3.1 - Gradient Descent 

In this section you will implement a function performing one step of stochastic gradient descent (with clipped gradients). You'll go through the training examples one at a time, so the optimization algorithm will be stochastic gradient descent. 

As a reminder, here are the steps of a common optimization loop for an RNN:

- Forward propagate through the RNN to compute the loss
- Backward propagate through time to compute the gradients of the loss with respect to the parameters
- Clip the gradients
- Update the parameters using gradient descent 

<a name='ex-3'></a>
### <font color='blue'> Exercise 3

Complete the implementation of the optimization process (one step of stochastic gradient descent).

Provided functions:

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ 
    Forward propagation through the RNN to compute the cross-entropy loss.
    
    Args:
    X -- Input data, list of integers representing characters' indices.
    Y -- Target data, list of integers representing characters' indices shifted one index.
    a_prev -- Previous hidden state.
    parameters -- Model parameters (Wax, Waa, Wya, b, by).
    
    Returns:
    loss -- Cross-entropy loss.
    cache -- Cached values for backpropagation.
    """
    # Implementation omitted
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    # Implementation omitted
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    # Implementation omitted
    return parameters
```

Recall that you previously implemented the `clip` function.

**Parameters**

* Note that the weights and biases inside the `parameters` dictionary are being updated by the optimization, even though `parameters` is not one of the returned values of the `optimize` function. The `parameters` dictionary is passed by reference into the function, so changes to this dictionary are making changes to the `parameters` dictionary even when accessed outside of the function.
* Python dictionaries and lists are "pass by reference". This means that if you pass a dictionary (or a list) into a function and modify it inside the function, you're actually modifying the original dictionary (or list), not a copy of it.

In [ ]:
def optimize(X, Y, a_prev, parameters, learning_rate=0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, each representing a character's index in the vocabulary.
    Y -- list of integers, same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- dictionary containing:
                    Wax -- Weight matrix for input-to-hidden, shape (n_a, n_x)
                    Waa -- Weight matrix for hidden-to-hidden, shape (n_a, n_a)
                    Wya -- Weight matrix for hidden-to-output, shape (n_y, n_a)
                    b -- Bias vector for hidden layer, shape (n_a, 1)
                    by -- Bias vector for output layer, shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- dictionary containing gradients:
                    dWax -- Gradient of input-to-hidden weights, shape (n_a, n_x)
                    dWaa -- Gradient of hidden-to-hidden weights, shape (n_a, n_a)
                    dWya -- Gradient of hidden-to-output weights, shape (n_y, n_a)
                    db -- Gradient of hidden layer bias, shape (n_a, 1)
                    dby -- Gradient of output layer bias, shape (n_y, 1)
    a[len(X)-1] -- last hidden state, shape (n_a, 1)
    """
    
    # Forward propagate through the RNN and compute the loss
    loss, cache = None
    
    # Backpropagate through the RNN to compute gradients
    gradients, a = None
    
    # Clip gradients between -5 (min) and 5 (max)
    gradients = None
    
    # Update parameters using the gradients and learning rate
    parameters = None
    
    return loss, gradients, a[len(X) - 1]


<a name='3-2'></a>
## <font color='darkviolet'> 3.2 - Training the Model 

<a name='ex-4'></a>
### <font color='blue'>  Exercise 4 

Complete the implementation of `model()`.

In [ ]:
def model(data_x, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
   
    """
    Trains the model and generates dinosaur names. 

    Arguments:
    data_x -- Text corpus, divided into words.
    ix_to_char -- Dictionary that maps the index to a character.
    char_to_ix -- Dictionary that maps a character to an index.
    num_iterations -- Number of iterations to train the model for.
    n_a -- Number of units of the RNN cell.
    dino_names -- Number of dinosaur names to sample at each iteration. 
    vocab_size -- Number of unique characters found in the text (size of the vocabulary).
    
    Returns:
    parameters -- Learned parameters.
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
        
    # Build list of all dinosaur names (training examples).
    examples = [x.strip() for x in data_x]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state 
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
                
        # Set the index `idx`into the list of examples
        idx = None
        
        # Extract a single example from the list of examples    
        X = None 
        
        # Get the integer representation of the newline character '\n'
        ix_newline = None
        
        # Set the list of labels (integer representation of the characters)
        Y = None

        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = None
                   
        # to keep the loss smooth.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters with sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            seed = 0
            # The number of dinosaur names to print
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix)
                last_dino_name = get_sample(sampled_indices, ix_to_char)
                print(last_dino_name.replace('\n', ''))
                seed += 1
                  
            print('\n')
    
    return parameters

When you run the following cell, you should observe your model outputting random-looking characters at the first iteration. After a few thousand iterations, your model should learn to generate reasonable-looking names. 

In [ ]:
parameters = model(data.split("\n"), ix_to_char, char_to_ix)

**Example of expected output**
```
...
Iteration: 22000, Loss: 22.728886

Onustreofkelus
Llecagosaurus
Mystolojmiaterltasaurus
Ola
Yuskeolongus
Eiacosaurus
Trodonosaurus
```

<font color='blue'> **Feel free to run the algorithm even longer and play with hyperparameters to see if you can get even better results! Your model hopefully learned that dinosaur names tend to end in saurus, don, aura, tor, etc.**